In [61]:
import pandas as pd
from PyPDF2 import PdfReader
import mysql.connector

reader = PdfReader("./data/reports/ORD-2025-101727.pdf")
page=reader.pages[0].extract_text()

print(page)#checking how extracted text from pdf looks like

Field Value
Warehouse WH-DXB-03
Sales Rep Elchin Martin
Customer Phone +994 589 580 3233
Customer VAT ID WM27591973
Packaging Box M
Dimensions (cm) 67×54×24
Volumetric Weight (kg) 17.37
Declared Value (USD) 13850.42
Last-Mile Partner PonyExpress
Signature Required Yes
QA Check Pass
Return Reason Address Issue
PDF Notes Contact customer before delivery.Shipment Dossier  ORD-2025-101727
T ennis Racket (SKU-SPO-17197)  Qty: 8
Customer: Nigar Huseynov  |  Segment: SMB
Destination: Baku, Azerbaijan  |  Carrier: DHL  |  Incoterm: CIF
Order Date: 2025-03-14    Ship Date: 2025-03-15    ETA: 2025-03-18
Shipment Code: SHP-1804350  |  Tracking: TRK-SHP-1804350-9503
External Document ID: DOC-101727-1804350
Generated: 2025-10-13 06:56:00 UTC Confidential  For Interview Use Only



In [62]:
pdfdf=pd.DataFrame(columns=['warehouse_code', 'sales_rep', 'customer_phone','customer_vat_id','packaging_type','package_l','package_w','package_h','volumetric_weight_kg',\
 'customs_declared_value_usd','last_mile_partner','signature_required','qa_check','return_reason','pdf_note','order_id','product_name','sku','quantity',\
 'customer_name','customer_segment','city','country','carrier','incoterm','order_date','ship_date','delivery_date','shipment_code','tracking_number','external_document_id'])

def process_page(page,pdfdf):#parsing the pdf
    lines=page.split('\n')
    
    warehouse_code=lines[1].split(" ")[1]
    sales_rep=" ".join(lines[2].split(" ")[2:])
    customer_phone="+"+lines[3].split("+")[1]
    customer_vat_id=lines[4].split(" ")[3]
    packaging_type=" ".join(lines[5].split(" ")[1:])
    package_l,package_w,package_h=lines[6].split(" ")[2].split("×")
    volumetric_weight_kg=lines[7].split(" ")[3]
    customs_declared_value_usd=lines[8].split(" ")[3]
    last_mile_partner=" ".join(lines[9].split(" ")[2:])
    signature_required=lines[10].split(" ")[2]
    qa_check=lines[11].split(" ")[2]
    return_reason=" ".join(lines[12].split(" ")[2:])
    pdf_note=" ".join(lines[13].split(" ")[2:-4])+" "+lines[13].split(" ")[-4].split(".")[0]+"."
    order_id=lines[13].split(" ")[-1]
    product_name=lines[14].split("(")[0]
    sku=lines[14].split("(")[1].split(")")[0]
    quantity=lines[14].split(" ")[-1]
    customer_name="".join(lines[15].split(":")[1].split("|")[0])
    customer_segment=lines[15].split(" ")[-1]
    city,country=lines[16].split(":")[1].split("|")[0].split(",")
    carrier=lines[16].split(":")[2].split(" ")[1]
    incoterm=lines[16].split(":")[3]
    order_date=lines[17].split(":")[1].split(" ")[1]
    ship_date=lines[17].split(":")[2].split(" ")[1]
    delivery_date=lines[17].split(":")[3].split(" ")[1]
    shipment_code=lines[18].split(":")[1].split(" ")[1]
    tracking_number=lines[18].split(":")[2].split(" ")[1]
    external_document_id=lines[19].split(":")[1]

    # print(sku)
    #putting the parsed data in dataframe
    pdfdf.loc[len(pdfdf)] = [warehouse_code,sales_rep,customer_phone,customer_vat_id,packaging_type,package_l,package_w,package_h,volumetric_weight_kg,customs_declared_value_usd,\
      last_mile_partner,signature_required,qa_check,return_reason,pdf_note,order_id,product_name,sku,quantity,customer_name,customer_segment,city,country,carrier,incoterm,\
         order_date,ship_date,delivery_date,shipment_code,tracking_number,external_document_id]
    return pdfdf

import os
base_dir = "./data/reports"

for root,folders,files in os.walk(base_dir):
    for file in files:
        reader = PdfReader(root+"/"+file)
        page=reader.pages[0].extract_text()
        pdfdf=process_page(page,pdfdf)
    
pdfdf.to_csv("pdf.csv",sep=";")
pdfdf

,warehouse_code,sales_rep,customer_phone,customer_vat_id,packaging_type,package_l,package_w,package_h,volumetric_weight_kg,customs_declared_value_usd,...,city,country,carrier,incoterm,order_date,ship_date,delivery_date,shipment_code,tracking_number,external_document_id
0,WH-NYC-01,Elchin Mammadov,+971 244 800 8209,HR61437969,Box L,20,42,36,6.05,14310.35,...,Munich,Germany,LocalPost,DDP,2024-03-19,2024-03-19,2024-03-23,SHP-3202660,TRK-SHP-3202660-3111,DOC-104052-3202660
1,WH-DXB-03,David Aliyeva,+33 691 579 6321,EZ31079609,Box L,22,10,20,0.88,218.11,...,Austin,USA,FedEx,CIF,2025-06-05,2025-06-05,2025-06-17,SHP-3265545,TRK-SHP-3265545-8720,DOC-101043-3265545
2,WH-TBS-01,Nina Martin,+994 578 826 1744,RP74214201,Poly Mailer,12,12,27,0.78,9880.23,...,Baku,Azerbaijan,LocalPost,CIF,2025-09-05,2025-09-09,2025-09-20,SHP-5250974,TRK-SHP-5250974-4973,DOC-103269-5250974
3,WH-DXB-03,Leyla Klein,+994 316 511 1263,RO41413883,Poly Mailer,60,58,4,2.78,465.41,...,Batumi,Georgia,FedEx,DAP,2023-03-10,2023-03-12,2023-03-30,SHP-7743485,TRK-SHP-7743485-8662,DOC-101240-7743485
4,WH-FRA-01,Nina Rahimov,+1 573 313 5299,NW70174510,Poly Mailer,65,14,6,1.09,10353.86,...,Moscow,Russia,LocalPost,DDP,2024-04-09,2024-04-11,2024-04-26,SHP-7409678,TRK-SHP-7409678-5275,DOC-101632-7409678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,WH-DXB-03,Murad Novak,+994 860 997 3543,LR28426343,Crate,37,47,3,1.04,6123.01,...,Tbilisi,Georgia,DHL,DAP,2024-05-23,2024-05-23,2024-06-07,SHP-7622766,TRK-SHP-7622766-4360,DOC-100343-7622766
4996,WH-TBS-01,Murad Novak,+49 747 129 5045,CE28374329,Box M,14,25,27,1.89,8998.88,...,Mumbai,India,DHL,DAP,2024-07-05,2024-07-07,2024-07-09,SHP-4813209,TRK-SHP-4813209-7176,DOC-101102-4813209
4997,WH-BAKU-01,Aysel Klein,+995 771 465 9870,XH29658498,Box M,53,32,12,4.07,8152.27,...,Ankara,Turkey,Aramex,DDP,2023-03-06,2023-03-10,2023-03-12,SHP-1299016,TRK-SHP-1299016-6987,DOC-103617-1299016
4998,WH-FRA-01,Kamran Klein,+995 429 595 1264,HS07271125,Crate,49,16,32,5.02,1386.75,...,Manchester,UK,UPS,DDP,2024-04-27,2024-04-30,2024-05-05,SHP-2127285,TRK-SHP-2127285-2393,DOC-103968-2127285


In [63]:
csvdf=pd.read_csv("data/dataset.csv")
csvdf.columns

Index(['record_id', 'order_id', 'customer_code', 'customer_name',
       'customer_email', 'customer_segment', 'country', 'city',
       'product_category', 'product_name', 'sku', 'quantity', 'unit_price',
       'discount_pct', 'shipping_cost', 'weight_kg', 'order_date', 'ship_date',
       'delivery_date', 'delivery_status', 'carrier', 'incoterm',
       'payment_method', 'currency', 'exchange_rate_to_usd', 'gross_revenue',
       'cogs', 'profit', 'risk_score', 'is_fraud_suspected', 'shipment_code',
       'tracking_number', 'record_notes'],
      dtype='object')

In [64]:
fulldf=pd.merge(csvdf,pdfdf,on="order_id",how="outer", suffixes=('', '_duplicate'))#merging csv and pdf dataframes
for i in fulldf.columns:
    if "duplicate" in i:
        fulldf=fulldf.drop(columns=[i])#deleting duplicate columns
fulldf.to_csv("fulldf.csv",sep=";")
fulldf

,record_id,order_id,customer_code,customer_name,customer_email,customer_segment,country,city,product_category,product_name,...,package_w,package_h,volumetric_weight_kg,customs_declared_value_usd,last_mile_partner,signature_required,qa_check,return_reason,pdf_note,external_document_id
0,1,ORD-2025-100001,CUST-221958,Ethan Aliyev,ethan.aliyev@example.com,SMB,Turkey,Istanbul,Apparel,Jeans,...,21,32,7.80,2822.92,CityCourier,Yes,Pass,,Scan label quality before dispatch.,DOC-100001-7423388
1,2,ORD-2025-100002,CUST-164820,Aylin Martin,aylin.martin@example.com,Corporate,Turkey,Ankara,Sports,Bicycle Helmet,...,23,19,1.40,20421.53,LastMileX,Yes,Pass,,Scan label quality before dispatch.,DOC-100002-5981505
2,3,ORD-2025-100003,CUST-256730,Ethan Mammadov,ethan.mammadov@example.com,Enterprise,Azerbaijan,Ganja,Home,Coffee Maker,...,16,6,0.21,5705.78,UrbanShip,No,Pass,,Include printed invoice inside the package.,DOC-100003-7113790
3,4,ORD-2025-100004,CUST-628178,Liam Martin,liam.martin@example.com,SMB,France,Marseille,Electronics,Action Camera,...,38,27,8.41,15684.31,LastMileX,Yes,Pass,Damaged,Check lithium battery handling rules.,DOC-100004-3663046
4,5,ORD-2025-100005,CUST-778843,Liam Brown,liam.brown@example.com,Consumer,UAE,Dubai,Electronics,Smartphone,...,57,19,6.71,490.55,LastMileX,No,Pass,,Check lithium battery handling rules.,DOC-100005-5056518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,ORD-2025-104996,CUST-817606,Nigar Aliyev,nigar.aliyev@example.com,Enterprise,Georgia,Batumi,Home,Desk Lamp,...,17,8,1.39,13692.33,GoGo,Yes,Pass,Lost in Transit,Check lithium battery handling rules.,DOC-104996-7037772
4996,4997,ORD-2025-104997,CUST-695037,Aisha Williams,aisha.williams@example.com,Consumer,USA,Austin,Books,History Book,...,50,6,1.20,29664.99,UrbanShip,No,Pass,Address Issue,Contact customer before delivery.,DOC-104997-6938015
4997,4998,ORD-2025-104998,CUST-461749,Aisha Smith,aisha.smith@example.com,Corporate,Georgia,Batumi,Electronics,Action Camera,...,27,18,3.60,1130.57,GoGo,Yes,Pass,,Contact customer before delivery.,DOC-104998-8235331
4998,4999,ORD-2025-104999,CUST-351074,Elvin Williams,elvin.williams@example.com,Corporate,Azerbaijan,Shaki,Apparel,Jacket,...,22,10,2.68,18889.32,UrbanShip,Yes,Pass,,Do not stack more than two boxes.,DOC-104999-9836671


In [85]:
#creating table in the database and defining schema
create_table_query = """
CREATE TABLE IF NOT EXISTS ragtable (
    record_id int ,
    order_id char(100) ,
    customer_code char(100) ,
    customer_name char(100) ,
    customer_email char(100) ,
    customer_segment char(100) ,
    country char(100) ,
    city char(100) ,
    product_category char(100) ,
    product_name char(100) ,
    sku char(20) ,
    quantity int ,
    unit_price double ,
    discount_pct double ,
    shipping_cost double ,
    weight_kg double ,
    order_date date ,
    ship_date date ,
    delivery_date date ,
    delivery_status char(20) ,
    carrier char(100) ,
    incoterm char(100) ,
    payment_method char(100) ,
    currency char(10) ,
    exchange_rate_to_usd double ,
    gross_revenue double ,
    cogs double ,
    profit double ,
    risk_score smallint ,
    is_fraud_suspected tinyint(1) ,
    shipment_code char(100) ,
    tracking_number char(100) ,
    record_notes char(100) ,
    warehouse_code char(100) ,
    sales_rep char(100) ,
    customer_phone char(100) ,
    customer_vat_id char(100) ,
    packaging_type char(20) ,
    package_l double ,
    package_w double ,
    package_h double ,
    volumetric_weight_kg double ,
    customs_declared_value_usd double ,
    last_mile_partner char(100) ,
    signature_required char(10) ,
    qa_check char(10) ,
    return_reason char(100) ,
    pdf_note char(100) ,
    external_document_id char(20) 
)
"""
db_connection = mysql.connector.connect(#connecting to database
    host="localhost",         
    user="root",             
    password="******", 
    database="ragsystem" 
)
cursor = db_connection.cursor()
cursor.execute(create_table_query)

In [86]:
for i, row in fulldf.iterrows():#converting dataframe into SQL table and loading it to database
    sql_query="""INSERT INTO ragtable (record_id,order_id,customer_code,customer_name,customer_email,customer_segment,country,city,product_category,product_name,sku,quantity,unit_price,
        discount_pct,shipping_cost,weight_kg,order_date,ship_date,delivery_date,delivery_status,carrier,incoterm,payment_method,currency,exchange_rate_to_usd,gross_revenue,cogs,profit,
            risk_score,is_fraud_suspected,shipment_code,tracking_number,record_notes,warehouse_code,sales_rep,customer_phone,customer_vat_id,packaging_type,package_l,package_w,package_h,
                volumetric_weight_kg,customs_declared_value_usd,last_mile_partner,signature_required,qa_check,return_reason,pdf_note,external_document_id) VALUES 
        (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    values=(row['record_id'],row['order_id'],row['customer_code'],row['customer_name'],row['customer_email'],row['customer_segment'],row['country'],row['city'],\
            row['product_category'],row['product_name'],row['sku'],row['quantity'],row['unit_price'],row['discount_pct'],row['shipping_cost'],row['weight_kg'],\
            row['order_date'],row['ship_date'],row['delivery_date'],row['delivery_status'],row['carrier'],row['incoterm'],row['payment_method'],row['currency'],\
            row['exchange_rate_to_usd'],row['gross_revenue'],row['cogs'],row['profit'],row['risk_score'],row['is_fraud_suspected'],row['shipment_code'],row['tracking_number'],\
            row['record_notes'],row['warehouse_code'],row['sales_rep'],row['customer_phone'],row['customer_vat_id'],row['packaging_type'],row['package_l'],row['package_w'],\
            row['package_h'],row['volumetric_weight_kg'],row['customs_declared_value_usd'],row['last_mile_partner'],row['signature_required'],row['qa_check'],row['return_reason']\
                ,row['pdf_note'],row['external_document_id'])
    cursor.execute(sql_query, values)
db_connection.commit()
db_connection.close()